In [ ]:
# Dependencies
import tweepy
import time
import numpy as np 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())



In [ ]:
# Search for bot mention
def get_mention():
    return api.search(q="@NB_ED4ever perform voodoo on ")

In [ ]:
# bot return
mentions = get_mention()

# Store tweet
tweet = mentions["statuses"][0]["text"]
#print(tweet)

# Billing
requested_user = mentions["statuses"][0]["user"]["screen_name"]

In [ ]:
# Store tweet text
stored_tweet = []

# twitter account
target_users = []

# Parse out target user handle to analyze
try:
    stored_tweet = tweet.split(" perform voodoo on ")
    possible_target_accounts = stored_tweet[1].split()
    for user in possible_target_accounts:
        # Make sure no duplicate user
        if user not in target_users:
            target_users.append(user)
except:
    print("Not Enough Mana...")

#print(target_users)

In [ ]:
# List to hold sentiment
results_list = []

# Loop through all target users
for target in target_users:

    # Variable for holding the oldest tweet
    oldest_tweet = None
    
    counter = 0

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []


    # Loop through 5 times
    for x in range(5):

        # Run search around each tweet
        public_tweets = api.search(target, count=100, result_type="recent", max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Convert time stamps to readable format
            converted_time = datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            results_list.append({
                "User": target,
                "Tweet Timestamps": converted_time,
                "Compound": results["compound"],
                "Positive": results["pos"],
                "Negative": results["neg"],
                "Neutral": results["neu"],
                "Tweets Ago": counter
                })

            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1
            
            counter += 1

# Create a DataFrame using results_list and display
victim_df = pd.DataFrame().from_dict(results_list)
victim_df = victim_df[["User", "Tweet Timestamps", "Compound", "Positive", "Negative", "Neutral", "Tweets Ago"]]
victim_df.head(10)

In [ ]:
x_vals = victim_df["Tweets Ago"]
y_vals = victim_df["Compound"]
plt.plot(x_vals, y_vals, marker="o", color="purple", linewidth=0.5, alpha=0.8)

now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")
plt.title(f"Sentiment Analysis of {target} Tweets at {now}")
plt.xlim([x_vals.max(), x_vals.min()])
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")

plt.savefig("Images/voodoodust.png", bbox_inches = "tight")

In [ ]:
# update twitter account with analysis
api.update_with_media("Images/voodoodust.png", f"Here you go @{requested_user}, this will cost you 1 gummy bear")

In [ ]:
double_check = True                
while(double_check == True):
    get_mention()
    time.sleep(120)
    user_input = input("Would you like to continue? y/n")
    if user_input != "y":
        double_check = False